# Groupby and Arrest Data

In our merging exercises, we examined the relationship between county-level violent arrest totals and county-level drug arrest totals. In those exercises, you were given a dataset that provided you with county-level arrest totals. But that's not actually how the data is provided by the state of California. This week we will work with the *raw* California arrest data, which is not organized by county or even county-year. 

**(1)** Download the raw California arrest data from the State Attorney General's office [here](https://openjustice.doj.ca.gov/data) by scrolling down to the "Arrests" category and downloading the "Arrests - CSV, 5.8 MB" file. 

In [43]:
import pandas as pd
#arrests = pd.read_csv("https://data-openjustice.doj.ca.gov/sites/default/files/dataset/2019-06/OnlineArrestData1980-2018.csv")
arrests = pd.read_csv("/users/nick/downloads/OnlineArrestData1980-2018.csv")

In [44]:
arrests.head()

,YEAR,GENDER,RACE,AGE_GROUP,COUNTY,VIOLENT,PROPERTY,F_DRUGOFF,F_SEXOFF,F_ALLOTHER,F_TOTAL,M_TOTAL,S_TOTAL
0,1980,Male,Black,Under 18,Alameda County,505,1351,188,26,79,2149,2286,295
1,1980,Male,Black,Under 18,Butte County,3,5,0,0,0,8,7,0
2,1980,Male,Black,Under 18,Calaveras County,0,0,0,0,0,0,1,0
3,1980,Male,Black,Under 18,Contra Costa County,116,446,28,2,37,629,557,31
4,1980,Male,Black,Under 18,El Dorado County,0,1,0,1,0,2,6,0


In [45]:
arrests.RACE.value_counts()

White       26590
Hispanic    23767
Other       23328
Black       21188
Name: RACE, dtype: int64

In [46]:
arrests.AGE_GROUP.value_counts()

20 to 29       17438
30 to 39       17258
40 to 69       17082
Under 18       16305
18 to 19       16040
70 and over    10750
Name: AGE_GROUP, dtype: int64

## Learning the Group Structure of Your Data

**(2)** What is the unit of observation for this dataset? In other words, when row zero says that there were 505 arrests for `VIOLENT` crimes, what exactly is that telling you -- 505 arrests in 1980? 505 arrests in Alameda County?

### Testing Your Assumptions

It's important to be able to test whether the data you are working with really is organized the way you think it is, especially when working with groupby, so let's discuss how to check your answer to number 2 with `duplicated`. Consider the following data:

In [47]:
df = pd.DataFrame({'social_security_numbers': [111111111, 222222222, 222222222, 333333333, 333333333], 'second_column': ['a', 'a', 'a', 'a', 'b']})
df

,social_security_numbers,second_column
0,111111111,a
1,222222222,a
2,222222222,a
3,333333333,a
4,333333333,b


If we want to see if there are any duplicate rows in the dataset, we can use `.duplicated()`:

In [48]:
df.duplicated()

0    False
1    False
2     True
3    False
4    False
dtype: bool

As you can see, `.duplicated()` looks at each row, and returns `True` if it has seen the row it is looking at before. Note that it doesn't tag *all* the rows that look similar -- it treats the first instance of a row as unique, and only tags subsequent repitions are "duplicates" (You can change this behavior with keyword arguments if you want all rows tagged).

Duplicated can also be used to test for duplicates on a sub-set of rows. For example, if we want to test for rows with duplicate values of the variable `social_security_numbers`, we can type:

In [49]:
df.duplicated(['social_security_numbers'])

0    False
1    False
2     True
3    False
4     True
dtype: bool

Since `duplicated` is now only looking at the first column, the last row is now a duplicate (because 333333333 is duplicated), where when we considered all columns, it was not a duplicate (because the value in the second column varied. 

We can now pair `.duplicated()` with the `.any()` function to test for the presence of duplicates in your dataset, which is how we test if we really understand what constitutes a unique observation (i.e. if we think each row of our data is a unique person, then we shouldn't see any duplicated values of social security numbers, which are unique to each person in the United States). 

When you run `.any()` on an array of booleans, it returns a single value of `True` if *any* entries are `True`, and a single value of `False` if *no* entries are `True`. (You can also use `.all()` to test if all entries are false). 

Thus the command: `df.duplicated(['social_security_numbers'])` will return `False` if `social_security_numbers` uniquely idenfies every row in our dataset (since there are no duplicates)! If any rows are duplicated, then `social_security_numbers` doesn't uniquely identify our observations (i.e. each row does not represent a unique person):

In [50]:
df.duplicated(['social_security_numbers']).any()

True

This might feel backward, so you can also add a `not` before the test if you want. :) In fact, in my code I add an explicit test using the `assert` statement. The command `assert` says "if the thing that follows this is `True`, don't do anything; if it's False, raise an exception. So in my code, I often write:  

In [51]:
assert not df.duplicated(['social_security_numbers']).any()

AssertionError: 

(which in this case raises an exception! Because the rows *aren't* unique!)

**(3)** Use `duplicated` to test if the variables *you* think uniquely identify rows in your data really do uniquely identify rows. If you were wrong, update your beliefs!

In [56]:
assert not arrests.duplicated(['YEAR', 'GENDER', 'RACE', 'AGE_GROUP', 'COUNTY']).any()

**(4)** Once you have a handle on how the data looks now, please **collapse the data** to be one observation per county-year-racial group. 

**Hint:** Think carefully about the most appropriate aggregation function given the data we're working with!

In [57]:
arrests_collapsed = arrests.groupby(['YEAR', 'COUNTY'], as_index=False).sum()
arrests_collapsed.head()

,YEAR,COUNTY,VIOLENT,PROPERTY,F_DRUGOFF,F_SEXOFF,F_ALLOTHER,F_TOTAL,M_TOTAL,S_TOTAL
0,1980,Alameda County,4504,9377,3569,257,1651,19358,71342,2338
1,1980,Alpine County,1,9,2,1,1,14,119,1
2,1980,Amador County,10,59,12,3,6,90,305,1
3,1980,Butte County,306,683,149,20,207,1365,4504,3
4,1980,Calaveras County,52,130,83,3,29,297,679,3


This merges are an easy place to do things wrong, so I'd also recommend eye-balling your data to be sure you did things right!

In [59]:
arrests_collapsed = arrests_collapsed.sort_values(['YEAR', 'COUNTY'])
arrests_collapsed.head(15)

,YEAR,COUNTY,VIOLENT,PROPERTY,F_DRUGOFF,F_SEXOFF,F_ALLOTHER,F_TOTAL,M_TOTAL,S_TOTAL
0,1980,Alameda County,4504,9377,3569,257,1651,19358,71342,2338
1,1980,Alpine County,1,9,2,1,1,14,119,1
2,1980,Amador County,10,59,12,3,6,90,305,1
3,1980,Butte County,306,683,149,20,207,1365,4504,3
4,1980,Calaveras County,52,130,83,3,29,297,679,3
5,1980,Colusa County,28,89,9,2,10,138,655,9
6,1980,Contra Costa County,1596,3851,1089,74,929,7539,25788,590
7,1980,Del Norte County,74,183,46,5,49,357,1078,14
8,1980,El Dorado County,198,574,178,12,118,1080,3266,50
9,1980,Fresno County,2367,3855,804,105,719,7850,37907,1178


In [63]:
# Check all counties have same years with black entries:
assert (arrests_collapsed.groupby(['COUNTY'])['YEAR'].count() == 39).all()

In [64]:
census = pd.read_csv('/users/nick/dropbox/MIDS_Data_Prep/County_Demographics/census_ts_nominal_county.csv', encoding='latin-1')
census.head()


,GISJOIN,YEAR,STATE,STATEFP,STATENH,COUNTY,COUNTYFP,COUNTYNH,NAME,B18AA,...,CV4AA,CV4AB,CV4AC,CV4AD,CV4AE,CV4AF,CV4AG,CV4AH,CV4AI,CV4AJ
0,G0100010,1970,Alabama,1,10,Autauga County,1,10,Autauga,17511,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,G0100030,1970,Alabama,1,10,Baldwin County,3,30,Baldwin,48650,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,G0100050,1970,Alabama,1,10,Barbour County,5,50,Barbour,12134,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,G0100070,1970,Alabama,1,10,Bibb County,7,70,Bibb,9953,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,G0100090,1970,Alabama,1,10,Blount County,9,90,Blount,26205,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [65]:
census = census[census.STATE == "California"]
census = census[census.YEAR >= 1980]

In [66]:
census.columns

Index(['GISJOIN', 'YEAR', 'STATE', 'STATEFP', 'STATENH', 'COUNTY', 'COUNTYFP',
       'COUNTYNH', 'NAME', 'B18AA', 'B18AB', 'B18AC', 'B18AD', 'B18AE',
       'CV4AA', 'CV4AB', 'CV4AC', 'CV4AD', 'CV4AE', 'CV4AF', 'CV4AG', 'CV4AH',
       'CV4AI', 'CV4AJ'],
      dtype='object')

In [31]:
census.CV4AA.value_counts(dropna=False)

514559.0    1
265294.0    1
17996.0     1
103958.0    1
41887.0     1
           ..
744282.0    1
130795.0    1
244309.0    1
21154.0     1
884.0       1
Name: CV4AA, Length: 232, dtype: int64

In [67]:
census = census.drop(['GISJOIN', 'STATEFP', 'STATE', 'STATENH', 'COUNTYFP', 'COUNTYNH', 'NAME'], axis='columns')
census.sample(10)

,YEAR,COUNTY,B18AA,B18AB,B18AC,B18AD,B18AE,CV4AA,CV4AB,CV4AC,CV4AD,CV4AE,CV4AF,CV4AG,CV4AH,CV4AI,CV4AJ
6471,1990,Imperial County,73615,2622,1859,31207,NaN,31742.0,2272.0,3195.0,159.0,NaN,41873.0,350.0,799.0,28913.0,NaN
3357,1980,San Joaquin County,266886,19288,4474,21738,NaN,237799.0,18443.0,23765.0,631.0,NaN,29087.0,845.0,2447.0,34325.0,NaN
9608,2000,Contra Costa County,621490,88813,5830,183969,48714.0,549409.0,86851.0,109486.0,2636.0,32658.0,72081.0,1962.0,3803.0,73874.0,16056.0
12800,2010,Ventura County,565804,15163,8068,197342,36941.0,400868.0,13082.0,57841.0,1371.0,18589.0,164936.0,2081.0,7316.0,138882.0,18352.0
12764,2010,Madera County,94456,5629,4136,40344,6300.0,57380.0,5009.0,4430.0,649.0,2405.0,37076.0,620.0,2670.0,36731.0,3895.0
9635,2000,Sacramento County,783240,121804,13359,233704,71392.0,706655.0,118073.0,148459.0,3406.0,51016.0,76585.0,3731.0,7063.0,88135.0,20376.0
9606,2000,Calaveras County,36982,304,705,1222,1341.0,35465.0,300.0,965.0,40.0,1019.0,1517.0,4.0,123.0,799.0,322.0
6510,1990,Tehama County,45593,256,966,2810,NaN,43049.0,246.0,1174.0,32.0,NaN,2544.0,10.0,145.0,2425.0,NaN
3321,1980,Amador County,18469,177,299,85,NaN,17811.0,177.0,363.0,29.0,NaN,658.0,0.0,21.0,255.0,NaN
12792,2010,Solano County,210751,60750,3212,107273,31358.0,168628.0,58743.0,64134.0,1463.0,21020.0,42123.0,2007.0,3115.0,41773.0,10338.0


In [68]:
census.columns

Index(['YEAR', 'COUNTY', 'B18AA', 'B18AB', 'B18AC', 'B18AD', 'B18AE', 'CV4AA',
       'CV4AB', 'CV4AC', 'CV4AD', 'CV4AE', 'CV4AF', 'CV4AG', 'CV4AH', 'CV4AI',
       'CV4AJ'],
      dtype='object')

In [69]:
race_table_vars = census.columns[7:]
race_table_vars

Index(['CV4AA', 'CV4AB', 'CV4AC', 'CV4AD', 'CV4AE', 'CV4AF', 'CV4AG', 'CV4AH',
       'CV4AI', 'CV4AJ'],
      dtype='object')

In [70]:
for i in race_table_vars:
    census[i] = census[i].fillna(0)

In [71]:
census['total_population'] = (census.CV4AA + census.CV4AB + census.CV4AC + 
                              census.CV4AD + census.CV4AE + census.CV4AF + 
                              census.CV4AG + census.CV4AH + census.CV4AI + census.CV4AJ)

In [72]:
census.total_population.head()

3319    1105379.0
3320       1097.0
3321      19314.0
3322     143851.0
3323      20710.0
Name: total_population, dtype: float64

In [73]:
census = census[['YEAR', 'COUNTY', 'total_population']]
census.YEAR.value_counts()

2010    58
2000    58
1990    58
1980    58
Name: YEAR, dtype: int64

In [74]:
arrests_w_pop = pd.merge(arrests_collapsed, census, on=['COUNTY', 'YEAR'], how='left', validate='1:1', indicator=True)
arrests_w_pop._merge.value_counts()

left_only     2030
both           232
right_only       0
Name: _merge, dtype: int64

In [75]:
assert (arrests_w_pop._merge != 'right_only').all()
arrests_w_pop = arrests_w_pop.drop('_merge', axis='columns')

In [76]:
arrests_w_pop.head()

,YEAR,COUNTY,VIOLENT,PROPERTY,F_DRUGOFF,F_SEXOFF,F_ALLOTHER,F_TOTAL,M_TOTAL,S_TOTAL,total_population
0,1980,Alameda County,4504,9377,3569,257,1651,19358,71342,2338,1105379.0
1,1980,Alpine County,1,9,2,1,1,14,119,1,1097.0
2,1980,Amador County,10,59,12,3,6,90,305,1,19314.0
3,1980,Butte County,306,683,149,20,207,1365,4504,3,143851.0
4,1980,Calaveras County,52,130,83,3,29,297,679,3,20710.0


**(11)** Because the US Census occurs once every 10 years, we only have population data for once every 10 years. To fill in the gaps in our data, we can *interpolate* the values between each census wave. For example, if a county is 75% White in 1990 and 25% in 2000, we could infer it was likely about 50% White in 1995.

`pandas` offers an `interpolate` method that will do this for you, but `interpolate` just doesn't interpolations for one set of observations. In this case, however, we need to do our interpolations *within each group*, so you'll have to figure out how to use `interpolate` with groupby. (*Hint:* this is probably a job for `apply`). 

In [78]:
# Let's clean up a little
arrests_w_pop = arrests_w_pop[['YEAR', 'COUNTY', 'VIOLENT', 'F_DRUGOFF', 'total_population']]
arrests_w_pop.head()

,YEAR,COUNTY,VIOLENT,F_DRUGOFF,total_population
0,1980,Alameda County,4504,3569,1105379.0
1,1980,Alpine County,1,2,1097.0
2,1980,Amador County,10,12,19314.0
3,1980,Butte County,306,149,143851.0
4,1980,Calaveras County,52,83,20710.0


In [79]:
arrests_w_pop = arrests_w_pop.sort_values(['COUNTY', 'YEAR'], ascending=True)
arrests_w_pop['interpolated_total_population'] = arrests_w_pop.groupby('COUNTY')['total_population'].apply(lambda x: x.interpolate())
arrests_w_pop.head()

,YEAR,COUNTY,VIOLENT,F_DRUGOFF,total_population,interpolated_total_population
0,1980,Alameda County,4504,3569,1105379.0,1105379.0
58,1981,Alameda County,4699,3926,NaN,1122759.3
116,1982,Alameda County,4389,4436,NaN,1140139.6
174,1983,Alameda County,4500,5086,NaN,1157519.9
232,1984,Alameda County,3714,5878,NaN,1174900.2


In [80]:
# Check a few counties to make sure it worked!
arrests_w_pop[white_arrests_w_pop.COUNTY == 'Alameda County']

/Users/Nick/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


,YEAR,COUNTY,VIOLENT,F_DRUGOFF,total_population,interpolated_total_population
0,1980,Alameda County,4504,3569,1105379.0,1105379.0
58,1981,Alameda County,4699,3926,NaN,1122759.3
116,1982,Alameda County,4389,4436,NaN,1140139.6
174,1983,Alameda County,4500,5086,NaN,1157519.9
232,1984,Alameda County,3714,5878,NaN,1174900.2
290,1985,Alameda County,4199,7139,NaN,1192280.5
348,1986,Alameda County,4915,7720,NaN,1209660.8
406,1987,Alameda County,5137,10333,NaN,1227041.1
464,1988,Alameda County,5087,12574,NaN,1244421.4
522,1989,Alameda County,5551,14056,NaN,1261801.7


In [82]:
arrests_w_pop[arrests_w_pop.COUNTY == 'Los Angeles County']

,YEAR,COUNTY,VIOLENT,F_DRUGOFF,total_population,interpolated_total_population
18,1980,Los Angeles County,37964,29319,7477503.0,7477503.0
76,1981,Los Angeles County,37610,27427,NaN,7616069.1
134,1982,Los Angeles County,36648,28807,NaN,7754635.2
192,1983,Los Angeles County,33858,36565,NaN,7893201.3
250,1984,Los Angeles County,35303,44331,NaN,8031767.4
308,1985,Los Angeles County,35659,51015,NaN,8170333.5
366,1986,Los Angeles County,45254,61035,NaN,8308899.6
424,1987,Los Angeles County,46020,61713,NaN,8447465.7
482,1988,Los Angeles County,51021,69363,NaN,8586031.8
540,1989,Los Angeles County,59012,69667,NaN,8724597.9


In [86]:
arrests_w_pop = arrests_w_pop.drop('total_population', axis='columns')
arrests_w_pop = arrests_w_pop.rename({'interpolated_total_population': 'total_population'}, axis='columns')

In [88]:
arrests_w_pop.to_csv('/users/nick/github/MIDS_Data/UDS_arrest_data.csv', index=False)

In [89]:
arrests_w_pop.head()

,YEAR,COUNTY,VIOLENT,F_DRUGOFF,total_population
0,1980,Alameda County,4504,3569,1105379.0
58,1981,Alameda County,4699,3926,1122759.3
116,1982,Alameda County,4389,4436,1140139.6
174,1983,Alameda County,4500,5086,1157519.9
232,1984,Alameda County,3714,5878,1174900.2
